In [22]:
import numpy as np
import pandas as pd
#read 3d arra"trues.npy" from file
trues = np.load("trues_un.npy")
data = pd.read_csv("dataset_example/DMI/dataset/test/wind_data.csv",header=[0,1])
print(data.shape, trues.shape)

(149, 80) (64, 6, 1)


In [46]:
# try to fin if this sequence of numbers trues[0] is in data
print(trues[0][1][0])
# is 1.7840391 in data?
print(trues[0][1][0] in data)
print(5.4 in data)
trues

8.799999876995685
False
False


array([[[ 9.79999992],
        [ 8.79999988],
        [ 8.30000002],
        [ 7.39999995],
        [ 6.60000002],
        [ 7.29999998]],

       [[ 8.79999988],
        [ 8.30000002],
        [ 7.39999995],
        [ 6.60000002],
        [ 7.29999998],
        [ 8.00000011]],

       [[ 8.30000002],
        [ 7.39999995],
        [ 6.60000002],
        [ 7.29999998],
        [ 8.00000011],
        [ 7.90000013]],

       [[ 7.39999995],
        [ 6.60000002],
        [ 7.29999998],
        [ 8.00000011],
        [ 7.90000013],
        [ 7.90000013]],

       [[ 6.60000002],
        [ 7.29999998],
        [ 8.00000011],
        [ 7.90000013],
        [ 7.90000013],
        [ 8.00000011]],

       [[ 7.29999998],
        [ 8.00000011],
        [ 7.90000013],
        [ 7.90000013],
        [ 8.00000011],
        [ 8.00000011]],

       [[ 8.00000011],
        [ 7.90000013],
        [ 7.90000013],
        [ 8.00000011],
        [ 8.00000011],
        [ 8.30000002]],

       [[ 7.90000013

In [45]:
data['wind_speed'].values[0][0]

5.4

In [48]:
data.columns.levels[0]

Index(['air_pressure_at_sea_level', 'air_temperature', 'cos_deg',
       'max(wind_speed_of_gust PT10M)', 'relative_humidity', 'sin_deg', 'time',
       'wind_speed'],
      dtype='object')

In [50]:
import requests
import pandas as pd
import numpy as np
import os
from math import sin, cos, sqrt, atan2, radians, atan, tan


# Insert your own client ID here
client_id = '95dff942-392f-45d5-9b3c-3364db30c5cf'
def get_data(endpoint, parameters):
    r = requests.get(endpoint, parameters, auth=(client_id, ''))
    # Extract JSON data
    json = r.json()

    # Check if the request worked, print out any errors
    if r.status_code == 200:
        data = json['data']
        # print('Data retrieved from frost.met.no!')
    else:
        print('Error! Returned status code %s' % r.status_code)
        print('Message: %s' % json['error']['message'])
        print('Reason: %s' % json['error']['reason'])
        raise ValueError

    return data
def get_stations(coord_main, polygon):
    print('Starting get_stations')
    endpoint = 'https://frost.met.no/sources/v0.jsonld'

    polygon_str = str([str(loc).translate({ ord(c): None for c in ",[]" }) for loc in polygon])
    polygon_str = 'POLYGON((' + polygon_str.translate({ ord(c): None for c in "[]''" }) + '))'

    parameters = {
        'geometry': polygon_str
    }
    data = get_data(endpoint, parameters)

    # This will return a Dataframe with all of the observations in a table format
    df = pd.DataFrame()
    print(df.head())
    for i in range(len(data)):
        row = pd.Series()
        row['name'] = data[i]['name']
        row['id'] = data[i]['id']
        row['validFrom'] = data[i]['validFrom']
        coord_i = data[i]['geometry']['coordinates']
        row['lon'] = coord_i[0]
        row['lat'] = coord_i[-1]

        # Get the distance to the target coordinate:
        dist_tar = distance_coord(coord_main[::-1], coord_i[::-1])
        row['dist_tar'] = dist_tar
        df = df.append(row, ignore_index=True)
    df = df.sort_values('dist_tar').reset_index(drop=True)

    return df

pd.set_option('display.max_columns', None)

# Let's say that you want to find stations that are close to coord_main, but lie within the polygon 
# specified below. get_stations() will return the available stations in the specified polygon in 
# sorted order of which are closest to coord_main.
# This is useful before you start downloading. 
coord_main = [9.8, 58.1]
polygon = [
    [9., 57.],
    [9., 59.],
    [11., 59.],
    [11, 57.],
]

# Get stations inside "polygon" in sorted order of which are closest to "coord_main"
stations = get_stations(coord_main, polygon)

Starting get_stations
Empty DataFrame
Columns: []
Index: []


NameError: name 'distance_coord' is not defined